In [1]:
# RLest_20231215ver
# Cross Entropyで推定したる！！
import pandas as pd
import os
import datetime
import numpy as np
from datetime import timedelta 
import csv
import time
import math
from scipy.stats import norm
from scipy.optimize import minimize
import random

In [2]:
df_data = pd.read_csv('/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/20230130_0845_20sec.csv')
search_folder = "/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/split_files/group_3_1/group_2"
file_list = list(os.listdir(search_folder))

df_link = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/stanw_link_post.csv')
L = len(df_link)

df_node = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/stanw_node.csv')
df_ble = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_nw_detail/ble_nw.csv')

In [3]:
#######################
######### 準備 ########
#######################

###### 各linkのoとdの座標を入れておく配列 ###### 
link_loc_array = [] 
for i in range(len(df_link)):
    linkid = df_link.loc[i, 'linkid']
    O = df_link.loc[i, 'o']
    D = df_link.loc[i, 'd']

    x_o = df_node[df_node['nodeid'] == O]['x'].iloc[0]
    y_o = df_node[df_node['nodeid'] == O]['y'].iloc[0]
    z_o = df_node[df_node['nodeid'] == O]['floor'].iloc[0] 
    o_loc = np.array([x_o, y_o, z_o])

    x_d = df_node[df_node['nodeid'] == D]['x'].iloc[0]
    y_d = df_node[df_node['nodeid'] == D]['y'].iloc[0]
    z_d = df_node[df_node['nodeid'] == D]['floor'].iloc[0]
    d_loc = np.array([x_d, y_d, z_d])

    loc_tuple = (o_loc, d_loc)
    link_loc_array.append(loc_tuple)


###### 各ビーコンの座標を入れておく配列 ###### 
beacon_loc_array = []
for j in range(len(df_ble)):
    beacon_id = df_ble.loc[j, 'ID']
    x_j = df_ble[df_ble['ID'] == beacon_id]['x'].iloc[0]
    y_j = df_ble[df_ble['ID'] == beacon_id]['y'].iloc[0]
    z_j = df_ble[df_ble['ID'] == beacon_id]['floor'].iloc[0]
    
    beacon_loc = np.array([x_j, y_j, z_j])
    beacon_loc_array.append(beacon_loc) 


###### ビーコンとリンク線分の中点間距離を返す関数 ######
def dist_to_mid(p1, p2, x):
    p1 = np.array(p1)
    p2 = np.array(p2)
    x = np.array(x)

    segment = p2 - p1
    v1 = x - p1
    v2 = x - p2
    mid_point = (p1 + p2) / 2

    v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment # 垂直ベクトルを計算
    distance = np.linalg.norm(x - mid_point)
    
    return distance


###### d_arrayの用意 ###### # 各リンクの中点とbleビーコンとの距離配列
d_array = np.zeros((len(df_link), len(df_ble)))
for i in range(len(df_link)):
    p_o = link_loc_array[i][0] # o座標 # i=1の時
    p_d = link_loc_array[i][1] # d座標

    for j in range(len(df_ble)):
        x_ap = df_ble.loc[j, 'x']
        y_ap = df_ble.loc[j, 'y']
        z_ap = df_ble.loc[j, 'floor']
        p_ap = np.array([x_ap, y_ap, z_ap])

        d_array[i, j] = dist_to_mid(p_o, p_d, p_ap) # ijはlink i+1, ビーコンj+1の距離


###### 混雑度dens，maxRSSI，meanRSSI，検出回数を各ビーコン各timestepごとに入れた配列（ビーコン数23*タイムステップ数） ######
# 15分間のデータでやるならtimstep18secなら50step, 20secなら45step
dens_jt = np.zeros((23, 45), dtype = int)
max_jt = np.zeros((23, 45), dtype = int)
mean_jt = np.zeros((23, 45), dtype = int)
#count_jt = np.zeros((23, 45), dtype = int)

grouped = df_data.groupby(['ID', 'timestep']) # listにして渡す必要がある
df_list = [group.reset_index(drop=True) for name, group in grouped]

for group in df_list:
    beacon_id = int(group.loc[0, 'ID'])
    timestep = int(group.loc[0, 'timestep'])

    # ユニークMACアドレス数，最大&平均RSSI
    unique_mac_count = group['MAC'].nunique()
    max_rssi = group['RSSI'].max()
    mean_rssi = group['RSSI'].mean()
    # countはそのタイムステップでの全検出数つまりgroupの長さそのもの
    #count = len(group)

    # numpy配列の要素はシーケンスダ
    dens_jt[beacon_id-1, timestep-1] = unique_mac_count
    max_jt[beacon_id-1, timestep-1] = max_rssi
    mean_jt[beacon_id-1, timestep-1] = mean_rssi
    #count_jt[beacon_id-1, timestep-1] = count


####### リンク接続行列 ####### ### 階段リンクだけは有向にしたい
def Imaking(link_data): 
    n = len(link_data)
    I = np.eye(n) # 全リンクに滞在リンク付与
    
    for i in range(n):

        if (i < 25) or (i >=  65): # 階段リンク以外の場合→これまで通り
            O = link_data.loc[i, 'o'] 
            D = link_data.loc[i, 'd'] 
            for j in range(n):
                if ((link_data.loc[j, 'o'] == O) or (link_data.loc[j, 'o'] == D)) or (link_data.loc[j, 'd'] == O) or (link_data.loc[j, 'd'] == D): 
                    I[i, j] = 1

        if 25 <= i < 65: ### iが25~64の場合→階段リンクの場合
            O = link_data.loc[i, 'o'] # onode
            D = link_data.loc[i, 'd'] # dnode

            for j in range(n):
                if link_data.loc[j, 'o'] == D: # DをOnodeとするリンクを1にしている
                    I[i, j] = 1
                
                if link_data.loc[j, 'o'] == D or link_data.loc[j, 'd'] == D: # odいずれかがDと一致するリンク[からの]遷移は0
                    I[j, i] = 0
                
                if link_data.loc[j, 'o'] == O or link_data.loc[j, 'd'] == O: # odいずれかがOと一致するリンクへの遷移は0
                    I[i, j] = 0

    return(I)


###################################
####### リンク接続行列（有向） ####### 
def I_withd(link_data): # integratedを読み込む想定
    n = len(link_data)
    I = np.eye(n)
    for i in range(n):
        D = link_data.loc[i, 'D']
        for j in range(n):
            if link_data.loc[j, 'O'] == D: # DnodeをOnodeとするリンクを1にしている
                I[i, j] = 1
    return(I)
###################################
###################################


I = Imaking(df_link)

for i in range(26, 65, 2): # 階段には滞在リンク入れない
    I[i-1, i-1] = 0
    I[i, i] = 0

for i in range(25, 46): # 階段リンクに入ったら半分のところで折り返さない
    I[i, i+20] = 0
    I[i+20, i] = 0

## 階段リンクへの接続では

for i in range(66, 69): # 吸収リンクからは吸収リンクにしか接続しない
    I[i-1, :] = 0 
    I[i-1, i-1] = 1 

####### パラメタ&タイムステップ設定 #######
xdim = 2
x = np.zeros(xdim)
x[0] = -1
x[-1] = 0.8

## 観測パラメタ二つ用意→b[0], b[1]で与えて推定したい
sigma1 = 2.0 # BLEの減衰定数．推定したいのと，混雑に影響されそう
sigma2 = 15 # 是非とも推定
T = 11 # inputデータによる．10stepで最終リンクには辿り着く人たち，吸収に行くのに+1timestep要すると考えるとT=11が妥当か
# 20ステップ以内ならT=20


/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_19815/3214949639.py:49: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment # 垂直ベクトルを計算


In [4]:
###### 経路選択モデリングのための時空間プリズム ######
# 起点Oからの到達可能性指示変数io
Itlist = np.zeros((T, L, L)) ## これは各状態に対して与えるので遷移回数T-1よりひとつ多くてT成分考える（timestep数がTなら状態数はT）
II = np.copy(I)
Itlist[0, :, :] = np.eye(L) 

for ts in range(1, T): 
    Itlist[ts, :, :] = II 
    II = np.dot(II, I)
    II = np.where(II > 0, 1, 0) 

# 終点Dからの到達可能性指示変数id ## これもIlist同様の考え方
Ittlist = np.zeros((T, L, L))
for ts in range(T): 
    Ittlist[ts, :, :] = np.transpose(Itlist[T - ts - 1, :, :]) 

###### 吸収リンク ######
ddata = [66, 67, 68] # linkidで指定
D = len(ddata)

In [5]:
##################
###### 準備 ######
##################

saikyo_key = [1, 2, 3, 4, 5, 6]
yamate_key = [7, 8, 9, 10, 11, 12, 13]
discriminate_key = [19, 22, 23]
abs_key = [16, 20, 21]

#### od_list作成 #### # ビーコン列からODを特定（ちょっとよくないけどとりあえず）
od_list = pd.DataFrame(columns=['userid', 'o', 'd', 'abs', 'Ti'])

#print(f'filelistの長さは{len(file_list)}')
count_file = 0
for file_name in file_list:

    dfi = pd.read_csv(os.path.join(search_folder, file_name))   # file_path = os.path.join(search_folder, file_name)
    userid = int(file_name.split('.')[0])                       # userid = int(base_name)

    Ti = dfi['timestep'].nunique()                              # 個人の滞在時間 Tiまでにlast_linkにくる   
    grouped = dfi.groupby('timestep')
    dfi_list = [group.reset_index(drop=True) for name, group in grouped]  

    ### 各人の各時刻の最大スコアのbeacon列 ###
    max_beacon_list = [] 
    for t in range(Ti):
        #print(f'時刻は{t}')
        dfit = dfi_list[t] # 時刻tの全ログデータ
        ts = int(dfit.loc[0, 'timestep'])
        # print(f'useridは{userid}で時刻{ts}の時，dfitは{dfit}')

        captured_beacon = dfit.groupby('ID')
        captured_beacon_list = [group.reset_index(drop=True) for name, group in captured_beacon]
        tdict = {}

        for j in range(len(dfit)):
            #print(f'dfitの長さはuseridは{userid}で時刻{ts}の時の観測ログ数{len(dfit)}')
            jid = dfit.loc[j, 'ID'] # そのログデータのビーコン
            max_rssi = max_jt[jid-1, ts-1] # そのビーコンの時刻tsでのmax_rssi
            rssi = dfit.loc[j, 'RSSI'] # そのログの電波強度
            jscore = max_rssi / rssi # このログのスコア→これを辞書のjidのところに足すイメージ
            if jid not in tdict:
                tdict[jid] = 0
            tdict[jid] += jscore  

        jmax = max(tdict, key = tdict.get) 
        max_beacon_list.append(jmax) #### なんかここ空になってる，なんで→append使う時はa = a.append()ってしないでa.append()だけにするらしいわ
        # これはGPTに言われるまで知らなかった，，，

    ### Oの確定 ### 
    first_beacon = max_beacon_list[0]
    first_dist_list = d_array[:, first_beacon-1] 
    first_dist_dict = {index + 1: value for index, value in enumerate(first_dist_list)} # keyはlinkidになっている．max_beaconと各リンクの距離が入ってる

    if any(item in discriminate_key for item in max_beacon_list): # 埼京オンリーにする
        filtered_dict = {key: first_dist_dict[key] for key in first_dist_dict if key in saikyo_key}
        max_key = min(filtered_dict, key = filtered_dict.get) # これが初期リンクとなる．埼京ホームのうち，最もmax_beaconに近いリンク
    else:
        filtered_dict = {key: first_dist_dict[key] for key in first_dist_dict if key in yamate_key}
        max_key = min(filtered_dict, key = filtered_dict.get)

    a0 = int(max_key)       # linkid
        
    #### Dの確定 #### 
    last_beacon = max_beacon_list[-1]

    if last_beacon not in abs_key: 
        continue
        
    last_link = 0
    if last_beacon == 16: # node39
        last_link = 25 
        abs_link = 46
    elif last_beacon == 20: # node28
        last_link = 17
        abs_link = 47
    elif last_beacon == 21: # node29
        last_link = 18   
        abs_link = 48   
    else:
        continue

    ## ここまで条件を満たせばod_listにデータを追加
    od_list.loc[count_file, 'userid'] = userid
    od_list.loc[count_file, 'o'] = a0
    od_list.loc[count_file, 'd'] = last_link
    od_list.loc[count_file, 'abs'] = abs_link
    od_list.loc[count_file, 'Ti'] = Ti

    count_file += 1

OD = len(od_list)

#print(f'od_listは作り終わってその長さは{OD}') # 長さは82→2fileだけ消去されているがほとんど生き残っている
#print(od_list)


###### 経路選択モデル用時空間プリズム ######
# od_listに各人のodと時間制約が入っているのでこれから各自の時空間プリズムを事前に準備（od_listは一定故関数化しなくていい）
#### T=10ならプリズムは最後last, T=11ならabs．
Ilist = []
for i in range(OD):   

    # OD確定読み込み
    ao = od_list.loc[i, 'o'] - 1  # index
    ad = od_list.loc[i, 'd'] - 1  # index
    aabs = od_list.loc[i, 'abs'] - 1
    userid = od_list.loc[i, 'userid']

    Id = np.zeros((T-1, L, L)) # Id[t]は時刻tの時の利用可能遷移k→aを示す．最後のt=T-1のとき不要なので要素数はt=0~T-2のT-1個

    for ts in range(T-1): # Tまでに吸収されるのでdはaabs．遷移回数はT-1なのでこれでOK
        if ts == 0:
            Id[ts, ao, :] = I[ao, :] # →全部利用可能なのおかしっくね？？と思うが一旦飲み込む．
            continue
        #print(f'----------userid{userid}，時刻{ts}のとき----------')
        #print('aaaaを出力します') #### 時刻ts=9のとき潰れる→そうか，Itlist[10]だめ？
        #print(f'Itlistの長さは？{Itlist.shape}')
        #aaaa = Itlist[ts + 1, ao, :] # Itlist[10]だめ？

        #print(f'aaaaは{aaaa}')
        #print('bbbbを出力します')
        #bbbb = Ittlist[ts + 1, aabs, :]
        #print(f'bbbbは{bbbb}')
        
        alist = np.where(Itlist[ts + 1, ao, :] == Ittlist[ts + 1, aabs, :])[0] ## ts=1（つまり第二回目の状態）のとき3番目の状態を見ている→OK
        ## Itlistの長さはT．最後はT-2で，そのときT-2+1=T-1, 
        for a in alist:
            if Itlist[ts + 1, ao, a] == Ittlist[ts + 1, aabs, a]: # always True
                klist = np.where(I[:, a] == 1)[0]

                for k in klist:
                    if len(np.where(Id[ts - 1, :, k] == 1)[0]) != 0:
                        Id[ts, k, a] = 1 

        #print(f'プリズムはこちら!{Id[ts, :, :]}') # ts=8（遷移回数9）で吸収のみになっている

    Ilist.append(Id)


###### 観測方程式 ######
def qjx(rssi, max_rssi, x_loc, j, b):               # bは観測誤差分散のパラメータ．x_loc（x, y, z)にいたときビーコンjでrssiの観測が得られる確率
    j_loc = beacon_loc_array[j-1]                   # id=jのビーコンの位置座標
    #s = max_rssi
    d_est = 10 ** ((max_rssi - rssi)/(10*b[0]))     # 減衰パラメタb[0] rssi0→-40にしてみる
    
    d = np.linalg.norm(x_loc - j_loc) 
    f = norm.pdf(d, d_est, b[1])                    # 誤差分散b[1]
    omega = (100 + rssi) / 10 
    # print(f'ビーコン{j}で強度{rssi}の場合d_estは{d_est}，位置{x_loc}とビーコン{j_loc}の距離dは{d}，観測確率は{omega * f}')

    return omega * f

In [14]:
def Mmodel(Pall, b): # 観測尤度を返す # xの代わりにPallを入れる．とにかくこれが本命！！！肝！！！！！！

    #print(f'経路選択パラメタ{x}と観測パラメタ{b}で観測モデル回す')
    count = 0 # 観測結果に入ったサンプル数（後で出力する）
    
    SLlist = [] 
    TLlist = []

    for od in range(OD):

        # odに対応するuserのファイルを読み込む（od_listのuserid列が実データとod_listを紐づけている）
        userid = od_list.loc[od, 'userid']
        file_name = f'{userid}.csv'
        dfi = pd.read_csv(os.path.join(search_folder, file_name)) 
        grouped = dfi.groupby('timestep')
        dfi_list = [group.reset_index(drop=True) for name, group in grouped]  

        #print(f'----************-----いまuserdi{userid}をみている-----************-----')

        Ti = od_list.loc[od, 'Ti'] # 個人odの観測限界．時刻Tiでlast_linkに到達し，時刻Ti+1で吸収される
        ao = od_list.loc[od, 'o'] - 1  # index
        ad = od_list.loc[od, 'd'] - 1  # index
        aabs = od_list.loc[od, 'abs'] - 1

        #### プリズム用意 #### odを与えたらプリズム制約．時刻Ti+1つまりTi回の遷移でaoからabsまで行く．もしくは時刻TiつまりTi-1回の遷移でaoからlastまで行く
        Idm = np.zeros((Ti, L, L)) ## 1成分目の値が遷移回数．Tiにするならabsまで行くことを仮定している．そのときItimlistは状態数＝時間数なのでTi+1で良い
        Itmlist = np.zeros((Ti+1, L, L))
        IIm = np.copy(I)
        Itmlist[0, :, :] = np.eye(L)

        for tt in range(1, Ti+1): 
            Itmlist[tt, :, :] = IIm 
            IIm = np.dot(IIm, I) 
            IIm = np.where(IIm > 0, 1, 0)

        Ittmlist = np.zeros((Ti+1, L, L))
        for tt in range(Ti+1): 
            Ittmlist[tt, :, :] = np.transpose(Itmlist[(Ti+1) - tt - 1, :, :])

        for tt in range(Ti):
            if tt == 0:
                Idm[tt, ao, :] = I[ao, :] 
                continue

            alist = np.where(Itmlist[tt + 1, ao, :] == Ittmlist[tt + 1, aabs, :])[0] # ここadじゃなくてaabsが適切では????
            for a in alist:
                if Itmlist[tt + 1, ao, a] == Ittmlist[tt + 1, aabs, a]: # ここadじゃなくてaabsが適切では????
                    klist = np.where(I[:, a] == 1)[0] 

                    for k in klist:
                        if len(np.where(Idm[tt - 1, :, k] == 1)[0]) != 0:
                            Idm[tt, k, a] = 1 ### 時刻tにおいて，a(t)→a(t+1)への遷移がOKなところが1になっている

        #print(f'時刻{tt}における観測プリズムは{Idm[tt, :, :]}') 
                            
        ####### 観測プリズムは実装できてるぽい ########

        SLmatrix = np.zeros((T, L)) # status_likelihood 状態数は遷移数+1
        TLmatrix = np.zeros((T-1, L, L)) # trans likelihood
        pmmatrix = np.zeros((T-1, L, L))  # p(m|a)のmatrixのはず
        pmstmatrix = np.zeros((T-1, L, L)) # 同時発生尤度のマトリックス
        
        
        # t = 0ではaoに確定的に存在するとする ####### ←ここがまずよくない．
        SLmatrix[0, ao] = 1 
        
        for t in range(1, T): #### t=tでは，k=a(t-1)からa=a(t)への遷移を考える．TLmatrixは遷移数＝T-1個しかないのでindexは0 ~ t-2

            #print(f'--**--いまuserid{userid}時刻{t}の状態及び時刻{t-1}から{t}への遷移を見ている--**--') 

            #####################################
            ######### 遷移発生尤度を計算 ##########
            #####################################

            if t <= Ti - 1:

                dfit = dfi_list[t]
                ts = int(dfit.loc[0, 'timestep'])

                it_grouped = dfit.groupby('ID')
                dfit_list = [it_group.reset_index(drop=True) for name, it_group in it_grouped]  

                slsum = 0
                nlist = np.argsort(SLmatrix[t-1])[::-1]  # tsにおける状態尤度が大きい順に並び替えたインデックスリスト
                ## t=1のときSL[0]を参照するがaoしかないのでそのままaoを見ることになる．t=2のときSL[1]を参照，tのときt-1を参照
                ## ループは1からまでにしてt-1における尤度が大きいリンクに対して??実装?
                ## t-1で尤度大きいリンクをkとしてtのリンクをaとして扱った方が実態に近い

                klist = [] # つまり時刻t-1で状態尤度が高い上位のリンクをklistに入れている

                for i in range(L):
                    slsum += SLmatrix[t-1, nlist[i]]
                    klist.append(nlist[i])
                    if slsum >= 1:
                        break
                
                klist3 = [i+1 for i in klist]
                
                #print(f'時刻{t-1}での状態発生尤度の高かったリンクは{klist3}') ##
                ###### どのリンクも尤度が低すぎてめちゃ大量にリンクが入ってる→解消している！！！！

                for k in klist: ## t-1での滞在尤度が高いリンクに対して走査

                    for a in range(L): ## tのリンクはこっち

                        ## 観測がないときは吸収リンクへの遷移確率が1でござるのでTLmatrix[t-1, aabs, aabs] = 1

                        if Idm[t-1, k, a] == 0: ### t-1時点でk→aが利用可能か否かを見る ###### 
                            TLmatrix[t-1, k, a] = 0 ### 多分これでいいのか？→利用不可なら尤度も0ということ
                            continue
                        
                        # a_loc = link_loc_array[a - 1] # kはidなので #### ここ間違ってた
                        a_loc = link_loc_array[a] ### こっちが正しい
                        a_o = a_loc[0]
                        a_d = a_loc[1]

                        ## 位置については一旦保留→aの上に確率的に存在するという仮定が望ましいが一旦リンクの中点にいるとする
                        x_loc = (a_o + a_d) / 2
                        
                        #print(f'リンク{a+1}のoは{a_o}，dは{a_d}，その中点は{x_loc}')

                        pm = 1
                        """
                        for j in range(len(dfit)): ### ここでようやく各観測ログにアプローチ
                            ### rssi小さい観測がpmの極小化を招いている説もある，積をとっているので．
                            ### でも相対評価で尤度最大化しようとしているならそこまで気にならないはずだが
                            j_id = dfit.loc[j, 'ID']
                            j_rssi = dfit.loc[j, 'RSSI']
                            j_max_rssi = max_jt[j_id - 1, ts - 1] 

                            pmpm = qjx(j_rssi, j_max_rssi, x_loc, j_id, b) # 観測方程式

                            #print(f'時刻{t}でリンクa(t)つまり{a+1}のビーコン{j_id}から電波強度{j_rssi}（maxは{j_max_rssi}）の観測確率は{pmpm}')

                            pm *= pmpm # qjx(j_rssi, j_max_rssi, x_loc, j_id, b) # 観測方程式
                        """
                        ######### 各ビーコンで最大強度の観測データのみ使う場合 #########
                        ###### ビーコンごとにgroupbyしてその中でrssiが最大のログに対してpmを計算する
                        for j in range(len(dfit_list)):
                            dfitj = dfit_list[j]
                            j_id = dfitj.loc[0, 'ID']
                            j_max_rssi = max_jt[j_id - 1, ts - 1] 
                            # dfitの中で，RSSI列が最大の要素のRSSIの値を取得
                            j_rssi = dfitj['RSSI'].max()

                            pm *= qjx(j_rssi, j_max_rssi, x_loc, j_id, b) # 観測方程式
          
                        pmmatrix[t-1, k, a] = pm # 時刻tの観測尤度はリンクaの観測確率ね→ただしindexの関係でt-1に入れる．つまりt=1にat-1=a0=k→at=a1=aの観測尤度がpm[0]に入ってる
                        #print(f'user{userid}時刻{t}でリンクa(t)つまり{a+1}のリンク観測尤度は{pm}')
                        ### ここがちゃんと出ているかは確認したほうがいい

                
                # 遷移尤度のnume pmst 時刻tにおいてk(t-1)→a(t)への遷移が観測された確率がt-1に入ってる
                # 時刻tにおいてk(t-1)→a(t)への遷移確率がPall[t-1]に入っている=時刻t-1にk(t-1)→a(t)に遷移する確率がPall[t-1]ということ
                # つまり時刻tにおけるat→at+1への遷移確率はPall[t]に入っているということか．で，aT-1→aTへの遷移確率まで入っているがaT-2→aT-1までしか使わない
                pmstmatrix[t-1, :, :] = pmmatrix[t-1, :, :] * Pall[od, t-1, :, :] # 観測確率pm*遷移確率（正規化前の観測尤度）

                befodeno_nonzelements = []

                # pmstmatrix[t-1, :, :]から値が0でない要素を見つけてリストに追加
                for row in pmstmatrix[t-1, :, :]:
                    for element in row:
                        if element != 0:
                            befodeno_nonzelements.append(element)


                # 正規化
                deno = np.sum(pmstmatrix[t-1, :, :], axis=1) # np.sum(pmstmatrix[t-1], axis=1)になってた
                deno = (deno == 0) * 1 + (deno != 0) * deno ## 観測尤度の分母
                
                deno_rep = np.tile(deno, L).reshape(L, L).T

                #print(f'~~~~~~~正規化部分~~~~~~~deno={deno}')
                pmstmatrix[t-1, :, :] = pmstmatrix[t-1, :, :] / deno_rep ## これが観測尤度

                afdeno_nonzelements = []

                # pmstmatrix[t-1, :, :]から値が0でない要素を見つけてリストに追加
                for row in pmstmatrix[t-1, :, :]:
                    for element in row:
                        if element != 0:
                            afdeno_nonzelements.append(element)
                #print(f'正規化前{befodeno_nonzelements}')
                #print(f'正規化後{afdeno_nonzelements}')


                # deno_rep = np.tile(deno, 3).reshape(3, 3).T

                # 非ゼロ要素のインデックスを取得
                #non_zero_indices1 = np.transpose(np.nonzero(pmstmatrix[t-1, :, :])) +1

                # タプル形式のリストに変換
                #non_zero_tuples1 = [tuple(index) for index in non_zero_indices1]

                # 結果の表示
                #print(f'時刻{t}での正規化後観測尤度のうち非零なリンク組み合わせ{non_zero_tuples1}')

                #### これも問題なさそう！！！

                #########################
                ######## 問題箇所 ########
                #########################

                sl = SLmatrix[t-1, :] # このままでは行ベクトル
                sl_col = sl[:, None]

                TLmatrix[t-1, :, :] = pmstmatrix[t-1, :, :] * sl_col # 同時発生尤度を計算してる（SLmatrixはindexが1増えてるので，同じt-1でもひとつ前の段階を指名している
                # 時刻tにk(a_t-1)からa(a_t)への遷移が発生する尤度
                #print(f'sl_colは{sl_col}')

                # 非ゼロ要素のインデックスを取得
                non_zero_indices = np.transpose(np.nonzero(TLmatrix[t-1, :, :])) +1

                # タプル形式のリストに変換
                non_zero_tuples = [tuple(index) for index in non_zero_indices]

                # 結果の表示
                #print(f'時刻{t}での状態遷移発生尤度のうち非零なリンク組み合わせ{non_zero_tuples}')
                ### これが潰れていたら，ほぼ確でSLmatrixのせいだ
                ########## →やっぱり潰れてる
                ################ pythonとRの文法の異なる部分．SLmatrix[t-1, :]を行ベクトルのままではなく列ベクトルとして計算しないといけない


            elif t == Ti:
                TLmatrix[t-1, ad, aabs] = 1 ### それ以外はそのまま→0のまま
            
            else:
                TLmatrix[t-1, aabs, aabs] = 1 ### それ以外はそのまま→0のまま


            ###################################
            ########## 状態発生尤度を定義 ########
            ###################################
    
            if t <= Ti - 2: # 観測プリズム（ao, ad, Tiから決定）Idm内の状態しか取り得ない．k(a_t-1)からa(a_t)への遷移がOKのときIdmt[t-1]が1

                dfit = dfi_list[t] # t<len(dfit)=Tiにおいて指定可能
                ts = int(dfit.loc[0, 'timestep'])

                for i in range(L): ## iは遷移先リンク，つまりa(t)の候補ということになる

                    klist = np.where(Idm[t-1, :, i] == 1)[0] ## 前時刻参照．時刻tにおいてリンクi=a(t)に遷移可能な前リンクk=a(t-1)を探し，klistに入れている．この時，構造化NWと観測情報からリンク候補はかなり絞られている

                    if len(klist) == 0:
                        continue

                    kTL = 0
                    for k in klist: # このiに移動できる元リンク（t-1時点でのリンクa(t-1)）
                        kTL += TLmatrix[t-1, k, i] ## 時刻t-1にk=a(t-1)からi = a(t)への遷移発生尤度をkTLに足していき，これがtでのiの存在尤度になる（それはそう！！）
                        #print(f'状態遷移 k = a(t-1) = {k+1} → a = a(t) = {i+1}（全部リンクid）の遷移発生尤度は{kTL}')
                        ##################################
                        ###### これがかなり0になっている！！
                        ##################################

                    SLmatrix[t, i] = kTL ### SLmatrixはindex t に入れる（SL[0]を事前に定義済みなので）

            if t == Ti - 1: # この時刻で最終リンクに確定的に滞在
                #print(f'時刻{t}で最後のリンク')
                SLmatrix[t, ad] = 1

            elif t >= Ti: # この時刻以降は確定的に吸収に滞在
                #print(f'時刻{t}で吸収リンク')             
                SLmatrix[t, aabs] = 1
            
            

        #SLlist[od] = SLmatrix
        SLlist.append(SLmatrix)
        #print(f'userid{userid}の状態発生尤度は{SLmatrix}')
        #TLlist[od] = TLmatrix
        TLlist.append(TLmatrix)
        #print(f'userid{userid}の状態遷移発生尤度は{TLmatrix}')
    
    return(TLlist) # 返すのは同時発生尤度で良さそう（石井2019と大山羽藤2017gpsの合体として立式）


In [7]:
###########################
####### 経路選択モデル ######
###########################

###### 即時効用行列 ###### # 混雑内生性考えないならT次元は圧縮可能だが一応→時不変だし，経路長も一緒だし経路パラメタしか入れてないので全リンクの効用が等しくなるよ．
def Mset(x): 
    #Probs_T = Probs.T # Probsを読み込む（内生性バージョン）
    inst = np.zeros((T-1, L, L))
    for t in range(T-1):
        inst_t = np.exp(df_link['length']/10 * x[0]) # + Probs_T[:, t] * x[1])    # + df_link['staire'] * x[1]) # + df_link_integrated['staire_with_esc'] * x[3])
        inst_t = pd.concat([inst_t]*L, axis=1) 
        inst_t = inst_t.T
        inst_t_numpy = inst_t.values # DataFrameをNumPy配列に変換
        inst[t, :, :] = inst_t_numpy

    return inst

In [8]:
###### リンク選択確率計算 ###### # 山野さんコードにおけるnewVを吸収している
### が，厳密には意味が異なる気がする．newV分離させて構造推定した方が良さそう
def newPall(x):
    #print(f'経路選択パラメタ{x}と観測パラメタ{b}で観測モデル回す')
    # Probs = np.zeros((T, L)) ## 時間tでのリンクkの期待人数＝個人ごとの選択確率の総和
    Pall = np.zeros((OD, T-1, L, L)) # 個人ごと，時刻ごとの各リンク間遷移確率行列 遷移の回数なのでT-1．今回Tを状態の数としているので遷移数はT-1になる
    beta = x[-1]
    

    #### 以降個人ごとに処理 #### 
    for od in range(OD): 
        
        Id = Ilist[od] # すでに個人のプリズムはIlistで用意してある

        ##### 即時効用 #####
        M = np.zeros((T-1, L, L))
        for ts in range(T-1):
            Mts = Id[ts, :, :] * Mset(x)[ts, :, :] # 内生性考慮してないのでMsetは定常
            M[ts, :, :] = Mts

        ##### 価値関数 ##### ### ここで価値関数求解（newVに相当）が入っている
        z = np.ones((T, L))
        for t in range(T-1, 0, -1):

            ####
            zii = M[t-1, :, :] * (z[t, :] ** beta) 
            zi = zii.sum(axis = 1)
            z[t-1, :] = (zi==0)*1 + (zi!=0)*zi
        
        ##### 選択確率行列 ##### # 尤度計算は不要
        for t in range(T-1): # 選択確率は各時刻tごとに決まる
            for k in range(L):
                for a in range(L):
                    if M[t, k, a] == 0: # 接続条件を満たせなかった観測は排除（logzero回避）
                        continue # ここがbreakになってたのが癌だった

                    Pall[od, t, k, a] += np.exp(np.log(M[t, k, a]) + beta * np.log(z[t+1, a]) - np.log(z[t, k])) 

    return Pall # 個人ごとの各時刻ごとのリンク遷移確率の一覧がPallに入った

In [9]:
### 構造推定対応として書き換える
###### 価値関数更新 ###### # 状態価値関数なので状態の数＝Tだけある
# 山野さんのコードは遷移回数がTなので状態数はT+1
# 今回のコードは状態数がTなので遷移回数はT-1
def newV(x):
    beta = x[-1]
    z = np.ones((T, L, OD)) # なんでODが第三成分なのかがよくわからないが
    V = np.zeros((T, L, OD))

    for od in range(OD):
        print(od_list.loc[od, 'userid'])
        Id = Ilist[od]
        #zI = np.ones((T, L))
        #VI = np.ze
        ##### 即時効用 #####
        M = np.zeros((T-1, L, L))
        for ts in range(T-1):
            Mts = Id[ts, :, :] * Mset(x)[ts, :, :] # 内生性考慮してないのでMsetは定常
            M[ts, :, :] = Mts

        ##### 価値関数 ##### ### ここで価値関数求解（newVに相当）が入っている
        for t in range(T-1, 0, -1): # T-1から1までのT-1回

            ####
            zi = np.dot(M[t-1, :, :], (z[t, :, od] ** beta) )
            #zi = zii.sum(axis = 1)
            z[t-1, :, od] = (zi==0)*1 + (zi!=0)*zi
            V[t-1, :, od] = np.log(z[t-1, :, od])

    return z

In [ ]:
np.set_printoptions(threshold=np.inf)
x1 = np.array([-1.5, 0.8])
b1 = np.array([2, 15]) # sigma2=5の時はpm小さすぎたけど10にしたら結構マシになってきた．やはり結構ばらけるのか
## rssi0=max_rssiが一応
Pall = newPall(x1)
print(Mmodel(Pall, b1))
## SLmatrixの途中が出てない，ao, ad, aabsは出ているが，，

In [19]:
def loglikelihood(x, b): # bの扱い微妙？
    KL = 0
    Pt = newPall(x)

    # sigmaを与える？

    TLlist = Mmodel(Pt, b)
    for od in range(OD):
        userid = od_list.loc[od, 'userid']
        TLmatrix = TLlist[od]
        for t in range(T-1):
            KLit = 0
            pit = Pt[od, t, :, :] # 時刻tにk(a_t-1)からa(a_t)への遷移確率はpall[t-1]に入っている
            pit = (pit == 0) * 1 + (pit != 0) * pit

            for k in range(L): # 時刻tにa(t-1)からa(tへの遷移を考える)
                for a in range(L):
                    logpka = 0
                    if pit[k, a] == 0:
                        logpka = 0

                        continue

                    if TLmatrix[t, k, a] == 0:
                        continue

                    else:
                        p = pit[k, a]
                        q = TLmatrix[t, k, a]

                        # print(f'userid{userid}の時刻{t}での{k+1}から{a+1}への状態遷移発生尤度{TLmatrix[t, k, a]}と遷移確率{pit[k, a]}，かけたら{- TLmatrix[t, k, a] * np.log(pit[k, a])}')
                    
                    qlq = q * np.log(q)
                    qlp = q * np.log(p)
                    # TLmatrix[t, k, a] * logpka # 時刻tにk(a_t-1)からa(a_t)への遷移発生尤度はTLmatrix[t-1]に入っている
                    ## TLmatrixに観測の結果が入っているのでt>=Ti以降はリンクを確定的に扱っている
                    ##### t = Tiではlast→aabs, t >= Ti + 1ではaabs→aabs，でそれぞれ確定してTLmatrix[Ti, last, aabs] = 1, TLmatrix[t>= Ti+1, aabs, aabs] = 1
                    ##### なのでTLmatrixを与える時点でそうする

                    # KLit += (qlq - qlp)
                    KLit += qlp
                    # LL += (np.log(q) - np.log(p))
                    print(f'{userid}::時刻{t}：{k+1}→{a+1}の遷移発生尤度：{q}，遷移確率：{p}，情報量：{(qlq - qlp)}')
            KL += KLit
            
    return KL

In [20]:
x1 = np.array([-1.5,  0.8]) 
b1 = np.array([2, 15]) 
np.set_printoptions(threshold=np.inf)
print(loglikelihood(x1, b1))

1006::時刻0：7→7の遷移発生尤度：0.028448775495748795，遷移確率：0.36560496268383963，情報量：-0.07264247727004948
1006::時刻0：7→8の遷移発生尤度：0.9382487781908242，遷移確率：0.5916118070879333，情報量：0.4326869791915232
1006::時刻0：7→34の遷移発生尤度：0.03330244631342718，遷移確率：0.04278323022822723，情報量：-0.008342774045885223
1006::時刻1：7→7の遷移発生尤度：0.0007141509633959701，遷移確率：0.3700481254258108，情報量：-0.004463653461299179
1006::時刻1：7→8の遷移発生尤度：0.027734624532352828，遷移確率：0.6299518745741887，情報量：-0.08661417326169689
1006::時刻1：8→7の遷移発生尤度：0.008134177180664618，遷移確率：0.20275765987822905，情報量：-0.026159000401883983
1006::時刻1：8→8の遷移発生尤度：0.31589728439570786，遷移確率：0.3451647479029688，情報量：-0.027989991013075977
1006::時刻1：8→9の遷移発生尤度：0.46855127612551123，遷移確率：0.3451647479029688，情報量：0.14320037929985968
1006::時刻1：8→30の遷移発生尤度：0.14566604048894033，遷移確率：0.10691284431583384，情報量：0.04505489150429742
1006::時刻2：7→8の遷移発生尤度：0.008848328144060589，遷移確率：0.6561745926494553，情報量：-0.03810265623088022
1006::時刻2：8→8の遷移発生尤度：0.08398071006233401，遷移確率：0.33744885213865583，情報量：-0.1168026238545699

In [21]:
x1 = np.array([0,  0.8]) 
b1 = np.array([2, 15]) 
np.set_printoptions(threshold=np.inf)
print(loglikelihood(x1, b1))

1006::時刻0：7→7の遷移発生尤度：0.028448775495748754，遷移確率：0.36560496268383913，情報量：-0.0726424772700494
1006::時刻0：7→8の遷移発生尤度：0.938248778190824，遷移確率：0.5916118070879333，情報量：0.4326869791915228
1006::時刻0：7→34の遷移発生尤度：0.033302446313427214，遷移確率：0.04278323022822728，情報量：-0.008342774045885237
1006::時刻1：7→7の遷移発生尤度：0.0007141509633959689，遷移確率：0.370048125425811，情報量：-0.004463653461299174
1006::時刻1：7→8の遷移発生尤度：0.027734624532352786，遷移確率：0.6299518745741892，情報量：-0.08661417326169682
1006::時刻1：8→7の遷移発生尤度：0.008134177180664616，遷移確率：0.2027576598782289，情報量：-0.026159000401883972
1006::時刻1：8→8の遷移発生尤度：0.3158972843957078，遷移確率：0.34516474790296864，情報量：-0.027989991013075866
1006::時刻1：8→9の遷移発生尤度：0.4685512761255111，遷移確率：0.34516474790296864，情報量：0.1432003792998598
1006::時刻1：8→30の遷移発生尤度：0.1456660404889405，遷移確率：0.10691284431583394，情報量：0.04505489150429748
1006::時刻2：7→8の遷移発生尤度：0.008848328144060585，遷移確率：0.6561745926494551，情報量：-0.038102656230880216
1006::時刻2：8→8の遷移発生尤度：0.083980710062334，遷移確率：0.33744885213865533，情報量：-0.1168026238545698
1006:

In [22]:
x1 = np.array([-5,  0.8]) 
b1 = np.array([2, 15]) 
np.set_printoptions(threshold=np.inf)
print(loglikelihood(x1, b1))

1006::時刻0：7→7の遷移発生尤度：0.028448775495748743，遷移確率：0.36560496268384024，情報量：-0.07264247727004947
1006::時刻0：7→8の遷移発生尤度：0.9382487781908241，遷移確率：0.5916118070879354，情報量：0.43268697919151966
1006::時刻0：7→34の遷移発生尤度：0.033302446313427235，遷移確率：0.042783230228227456，情報量：-0.008342774045885362
1006::時刻1：7→7の遷移発生尤度：0.0007141509633959662，遷移確率：0.3700481254258101，情報量：-0.0044636534612991574
1006::時刻1：7→8の遷移発生尤度：0.027734624532352776，遷移確率：0.6299518745741898，情報量：-0.08661417326169682
1006::時刻1：8→7の遷移発生尤度：0.008134177180664583，遷移確率：0.2027576598782287，情報量：-0.026159000401883882
1006::時刻1：8→8の遷移発生尤度：0.3158972843957076，遷移確率：0.3451647479029694，情報量：-0.027989991013076865
1006::時刻1：8→9の遷移発生尤度：0.4685512761255109，遷移確率：0.3451647479029694，情報量：0.14320037929985846
1006::時刻1：8→30の遷移発生尤度：0.14566604048894097，遷移確率：0.1069128443158346，情報量：0.045054891504297256
1006::時刻2：7→8の遷移発生尤度：0.008848328144060549，遷移確率：0.6561745926494553，情報量：-0.0381026562308801
1006::時刻2：8→8の遷移発生尤度：0.08398071006233389，遷移確率：0.33744885213865405，情報量：-0.1168026238545694

In [23]:
x1 = np.array([-10,  0.8]) 
b1 = np.array([2, 15]) 
np.set_printoptions(threshold=np.inf)
print(loglikelihood(x1, b1))

1006::時刻0：7→7の遷移発生尤度：0.028448775495748552，遷移確率：0.3656049626838351，情報量：-0.07264247727004877
1006::時刻0：7→8の遷移発生尤度：0.9382487781908243，遷移確率：0.5916118070879312，情報量：0.4326869791915267
1006::時刻0：7→34の遷移発生尤度：0.03330244631342725，遷移確率：0.04278323022822715，情報量：-0.008342774045885099
1006::時刻1：7→7の遷移発生尤度：0.0007141509633959613，遷移確率：0.3700481254258127，情報量：-0.0044636534612991375
1006::時刻1：7→8の遷移発生尤度：0.027734624532352592，遷移確率：0.6299518745741942，情報量：-0.08661417326169664
1006::時刻1：8→7の遷移発生尤度：0.008134177180664596，遷移確率：0.2027576598782287，情報量：-0.02615900040188392
1006::時刻1：8→8の遷移発生尤度：0.315897284395708，遷移確率：0.3451647479029694，情報量：-0.027989991013076365
1006::時刻1：8→9の遷移発生尤度：0.4685512761255115，遷移確率：0.3451647479029694，情報量：0.14320037929985924
1006::時刻1：8→30の遷移発生尤度：0.14566604048894016，遷移確率：0.10691284431583384，情報量：0.0450548915042972
1006::時刻2：7→8の遷移発生尤度：0.008848328144060558，遷移確率：0.6561745926494553，情報量：-0.03810265623088012
1006::時刻2：8→8の遷移発生尤度：0.08398071006233396，遷移確率：0.33744885213865405，情報量：-0.11680262385456948
1006

In [24]:
x1 = np.array([-20,  0.8]) 
b1 = np.array([2, 15]) 
np.set_printoptions(threshold=np.inf)
print(loglikelihood(x1, b1))

1006::時刻0：7→7の遷移発生尤度：0.028448775495749694，遷移確率：0.36560496268384024，情報量：-0.07264247727005092
1006::時刻0：7→8の遷移発生尤度：0.9382487781908222，遷移確率：0.5916118070879144，情報量：0.43268697919155025
1006::時刻0：7→34の遷移発生尤度：0.033302446313428116，遷移確率：0.04278323022822715，情報量：-0.008342774045884446
1006::時刻1：7→7の遷移発生尤度：0.0007141509633959901，遷移確率：0.3700481254258075，情報量：-0.004463653461299279
1006::時刻1：7→8の遷移発生尤度：0.027734624532353702，遷移確率：0.6299518745741852，情報量：-0.0866141732616986
1006::時刻1：8→7の遷移発生尤度：0.008134177180664557，遷移確率：0.20275765987823158，情報量：-0.02615900040188395
1006::時刻1：8→8の遷移発生尤度：0.31589728439570663，遷移確率：0.3451647479029743，情報量：-0.027989991013082194
1006::時刻1：8→9の遷移発生尤度：0.4685512761255094，遷移確率：0.3451647479029743，情報量：0.1432003792998498
1006::時刻1：8→30の遷移発生尤度：0.14566604048894158，遷移確率：0.10691284431583688，情報量：0.045054891504294925
1006::時刻2：7→8の遷移発生尤度：0.008848328144060547，遷移確率：0.6561745926494645，情報量：-0.038102656230880216
1006::時刻2：8→8の遷移発生尤度：0.08398071006233418，遷移確率：0.33744885213865883，情報量：-0.1168026238545707

In [25]:
x1 = np.array([-30,  0.8]) 
b1 = np.array([2, 15]) 
np.set_printoptions(threshold=np.inf)
print(loglikelihood(x1, b1))

1006::時刻0：7→7の遷移発生尤度：0.02844877549574739，遷移確率：0.36560496268384024，情報量：-0.07264247727004736
1006::時刻0：7→8の遷移発生尤度：0.9382487781908261，遷移確率：0.5916118070879648，情報量：0.432686979191476
1006::時刻0：7→34の遷移発生尤度：0.03330244631342637，遷移確率：0.042783230228228365，情報量：-0.008342774045886708
1006::時刻1：7→7の遷移発生尤度：0.0007141509633959322，遷移確率：0.370048125425818，情報量：-0.004463653461298995
1006::時刻1：7→8の遷移発生尤度：0.02773462453235146，遷移確率：0.6299518745742032，情報量：-0.08661417326169463
1006::時刻1：8→7の遷移発生尤度：0.008134177180664592，遷移確率：0.2027576598782258，情報量：-0.02615900040188379
1006::時刻1：8→8の遷移発生尤度：0.31589728439570797，遷移確率：0.3451647479029645，情報量：-0.027989991013071924
1006::時刻1：8→9の遷移発生尤度：0.4685512761255113，遷移確率：0.3451647479029645，情報量：0.14320037929986557
1006::時刻1：8→30の遷移発生尤度：0.14566604048894216，遷移確率：0.10691284431583384，情報量：0.045054891504299865
1006::時刻2：7→8の遷移発生尤度：0.008848328144060525，遷移確率：0.656174592649446，情報量：-0.03810265623087988
1006::時刻2：8→8の遷移発生尤度：0.08398071006233339，遷移確率：0.3374488521386492，情報量：-0.11680262385456806
1006:

In [26]:
x1 = np.array([0, 0]) 
b1 = np.array([2, 15]) 
np.set_printoptions(threshold=np.inf)
print(loglikelihood(x1, b1))

1006::時刻0：7→7の遷移発生尤度：0.031862683833216845，遷移確率：0.3333333333333333，情報量：-0.07480426024566968
1006::時刻0：7→8の遷移発生尤度：0.6493996242278322，遷移確率：0.3333333333333333，情報量：0.4330880451822306
1006::時刻0：7→34の遷移発生尤度：0.31873769193895096，遷移確率：0.3333333333333333，情報量：-0.014271323373248213
1006::時刻1：7→7の遷移発生尤度：0.0013380353018676028，遷移確率：0.5，情報量：-0.007925726004809959
1006::時刻1：7→8の遷移発生尤度：0.030524648531349247，遷移確率：0.5，情報量：-0.08534916368239724
1006::時刻1：8→7の遷移発生尤度：0.007088576271653858，遷移確率：0.25，情報量：-0.02525643000399433
1006::時刻1：8→8の遷移発生尤度：0.1617119510807244，遷移確率：0.25，情報量：-0.07044888084228376
1006::時刻1：8→9の遷移発生尤度：0.2398575258048314，遷移確率：0.25，情報量：-0.009933894501477358
1006::時刻1：8→30の遷移発生尤度：0.2407415710706225，遷移確率：0.25，情報量：-0.009084835519997503
1006::時刻2：7→8の遷移発生尤度：0.008426611573521462，遷移確率：0.5，情報量：-0.034407652921018446
1006::時刻2：8→8の遷移発生尤度：0.03630690649631789，遷移確率：0.25，情報量：-0.07005246723046654
1006::時刻2：8→9の遷移発生尤度：0.07801622122325626，遷移確率：0.25，情報量：-0.09085333390531833
1006::時刻2：8→30の遷移発生尤度：0.0779134718924995，遷